In [3]:
"""
Converts output of MediaPipe (.json) to a .txt file using all 33 pose keypoints.
(Adapted from a script originally for OpenPose output)
creat_db.py is required to be run after this script to merge all text files into format required for RNN for Human Activity Recognition - (Now 33*2D Pose Input)

Original OpenPose script by Stuart Eiffert 13/12/2017
Modifications for MediaPipe (33 points) by Your Name/AI 2025/05/28

All code is provided under the MIT License (assuming original license applies)

"""


import json
from pprint import pprint # Keep for debugging if needed
import glob, os
import numpy as np # For handling missing keypoints

# --- Configuration based on your MediaPipe script ---
data_path = os.path.join(os.getcwd(), r'C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON')
#activity_list = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move'])
activity_list = np.array(['ageru', 'understand', 'annsinnsuru' , 'heavy'])
num_videos_per_action =15 #10#90 

fixed_cluster_num = 1
fixed_camera_num = 1
fixed_repetition_num = 1

# Number of keypoints from MediaPipe Pose (33 landmarks)
NUM_MEDIAPIPE_KEYPOINTS = 42 #12
NUM_OUTPUT_VALUES = NUM_MEDIAPIPE_KEYPOINTS * 2 # x and y for each keypoint

# Loop through actions (activities) and videos (subjects)
for activity_idx, activity_name in enumerate(activity_list):
    for video_num_loop in range(1, num_videos_per_action + 1):

        subject_num_str = str(video_num_loop)
        if video_num_loop < 10:
            subject_num_str = "0" + subject_num_str

        activity_num_str = str(activity_idx + 1)
        if activity_idx + 1 < 10:
             activity_num_str = "0" + activity_num_str

        frame_set = f"l0{fixed_cluster_num}_c0{fixed_camera_num}_s{subject_num_str}_a{activity_num_str}_r0{fixed_repetition_num}"
        current_video_json_path = os.path.join(data_path, activity_name, str(video_num_loop))

        if not os.path.isdir(current_video_json_path):
            print(f"Warning: Directory not found, skipping: {current_video_json_path}")
            continue

        os.chdir(current_video_json_path)

        kps_all_frames = []

        json_files = sorted(glob.glob("*.json"), key=lambda x: int(os.path.splitext(x)[0]))

        for file_name in json_files:
            with open(file_name) as data_file:
                data = json.load(data_file)

                frame_kps_mp33 = [] # For all 33 MediaPipe keypoints (66 values)

                mp_pose_landmarks = data.get("pose", [])

                if not mp_pose_landmarks:
                    # Fill with zeros if no pose landmarks are found
                    frame_kps_mp33 = [0.0] * NUM_OUTPUT_VALUES
                    # print(f"Warning: No pose landmarks in {file_name}. Filling with zeros.")
                else:
                    # Extract x, y for all 33 landmarks
                    for landmark_idx in range(NUM_MEDIAPIPE_KEYPOINTS):
                        if landmark_idx < len(mp_pose_landmarks):
                            landmark = mp_pose_landmarks[landmark_idx]
                            frame_kps_mp33.append(landmark.get('x', 0.0))
                            frame_kps_mp33.append(landmark.get('y', 0.0))
                        else:
                            # Should not happen if mp_pose_landmarks has 33 elements
                            # or if we are iterating up to len(mp_pose_landmarks)
                            # However, to be safe, if a landmark is missing for some reason before 33.
                            frame_kps_mp33.extend([0.0, 0.0])

                    # Ensure we always have NUM_OUTPUT_VALUES values
                    # This is important if mp_pose_landmarks had less than 33 elements
                    # (though MediaPipe typically returns all 33 or none)
                    if len(frame_kps_mp33) < NUM_OUTPUT_VALUES:
                        frame_kps_mp33.extend([0.0] * (NUM_OUTPUT_VALUES - len(frame_kps_mp33)))
                    elif len(frame_kps_mp33) > NUM_OUTPUT_VALUES: # Should not happen with current logic
                        frame_kps_mp33 = frame_kps_mp33[:NUM_OUTPUT_VALUES]


                kps_all_frames.append(frame_kps_mp33)

        output_dir_for_action = os.path.join(data_path, activity_name)
        if not os.path.isdir(output_dir_for_action):
             os.makedirs(output_dir_for_action)

        output_file_name = f"{activity_name}{frame_set}_mp33.txt" # Added _mp33 to distinguish
        output_file_path = os.path.join(output_dir_for_action, output_file_name)

        with open(output_file_path, "w") as text_file:
            for frame_data in kps_all_frames:
                if len(frame_data) != NUM_OUTPUT_VALUES:
                    print(f"Error: Frame data for {output_file_name} has {len(frame_data)} values, expected {NUM_OUTPUT_VALUES}. Skipping frame or writing as is.")
                    # Decide how to handle this: skip, pad, or write as is. For now, writing as is.
                
                for i, val in enumerate(frame_data):
                    text_file.write('{}'.format(val))
                    # Adjust comma separation for NUM_OUTPUT_VALUES
                    if i < (NUM_OUTPUT_VALUES - 1):
                        text_file.write(',')
                text_file.write('\n')
        print(f"Successfully wrote MediaPipe 42-point data for {activity_name} video {video_num_loop} to {output_file_path}")

Successfully wrote MediaPipe 42-point data for ageru video 1 to C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON\ageru\agerul01_c01_s01_a01_r01_mp33.txt
Successfully wrote MediaPipe 42-point data for ageru video 2 to C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON\ageru\agerul01_c01_s02_a01_r01_mp33.txt
Successfully wrote MediaPipe 42-point data for ageru video 3 to C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON\ageru\agerul01_c01_s03_a01_r01_mp33.txt
Successfully wrote MediaPipe 42-point data for ageru video 4 to C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON\ageru\agerul01_c01_s04_a01_r01_mp33.txt
Successfully wrote MediaPipe 42-point data for ageru video 5 to C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\MP_Data_JSON\ageru\agerul01_c01_s